In [1]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import time
import sys

In [2]:
# Now we get to market simulation 

# choose your number of periods
T = 1

# Initial stock price
s0 = 4

# fix the parameters in the fixed-interest rate, fixed-volatility setting
r = 0.25 # interest rate
u = 2. # up jump
d = 0.5 # down jump

# TODO: automatic check for arbitrage
if d >= 1+r or u <= 1+r:
    print('Arbitrage detected in the market! Derivative pricing models invalid.')

# Use this data to define the risk-neutral probabilities
p_rn = (1.+r-d)/(u-d)
q_rn = (u-1.-r)/(u-d)

We define the graph that organizes our thought process using the NetworkX package. 

In [3]:
# define the t^th triangular number... tri(0)=0, tri(1)=1, tri(2)=3,...
# note: this only works for SCALAR argument bcz of the way python's "int" fnc works...
def tri(t):
    return  int(0.5*t*(t+1))

In [4]:
# initialize the graph as "empty" (vertices only) with the right num of vertices
nverts = tri(T+1)
G = nx.empty_graph(nverts) 

# set the period attribute of the root node 
nx.set_node_attributes(G, {0:{'period':0}})

# set price attribute of the root node
nx.set_node_attributes(G, {0:{'s':s0}})

for t in range(0,T): 
    
    # triangular numbers can be used for indexing help
    numt = tri(t)
    
    for n in range(numt, numt+t+1):
                
        # set period attribute for next period
        nx.set_node_attributes(G, {n+t+1:{'period':t+1}})
        nx.set_node_attributes(G, {n+t+2:{'period':t+1}})
        
        # set underlying price, s, attribute for next period
        s = G.nodes[n]['s']
        
        s_u = u*s
        s_d = d*s
        nx.set_node_attributes(G, {n+t+1:{'s':s_u}})
        nx.set_node_attributes(G, {n+t+2:{'s':s_d}})
        
        # TODO: currently there is some redundancy with populating s! Fix this. 
        
        # draw new edges (since we only draw edges, no worries about double-counting
        # which would happen if we drew nodes instead)
        G.add_edge(n, n+t+1)
        G.add_edge(n, n+t+2)

In [5]:
# draw the graph if you want, but right now the visual presentation isn't optimized
#nx.draw_networkx(G)
#G.nodes(data=True)
#G.nodes[1]['s']

In [6]:
# risk-neutral derivative pricing formula if underlying price is s and upward 
# forward derivative price is v_u, downward forward price is v_d.
def rn_pricing_formula(v_u, v_d):
    
    return (1./(1.+r))*(p_rn*v_u + q_rn*v_d) # compute discounted expectation in the RN measure

In [7]:
# delta-hedging formula
def delta_hedging(s, v_u, v_d):

    return (v_u - v_d)/((u-d)*s)

In [8]:
# price a derivative security for sale at time t0, given only the derivative's payoff at time T
# as a function of underlying price s(T). Also populate replicating portfolio deltas and
# bond balances
def price_derivative(t0, payoff): 
    
    # TODO: print error message if sale time t0 is execution time T
    
    for t in range(T, -1, -1):
        
        numt = tri(t)
        
        for n in range(numt, numt+t+1):
        
            node = G.nodes[n]
            
            s = node['s']
                
            # first compute the payoff on the final period, and populate vertex attributes with the results
        
            if t == T:
                                        
                v = payoff(s)
                
                delta = 0. # have no info on delta at the terminal time! 
                
                M = 0. # likewise have no info on ideal money market balance at terminal time!
            
            # now do recursion down the graph
        
            else: 
            
                v_u = G.nodes[n+t+1]['v']
        
                v_d = G.nodes[n+t+2]['v']
            
                v = rn_pricing_formula(v_u, v_d)
                
                delta = delta_hedging(s, v_u, v_d)
                
                M = v - delta*s
            
            nx.set_node_attributes(G, {n:{'v':v}})
            nx.set_node_attributes(G, {n:{'delta':delta}})
            nx.set_node_attributes(G, {n:{'M':M}})
        
    # extract the derivative price at whatever chosen time
    v0 = G.nodes[t0]['v']
    delta0 = G.nodes[t0]['delta']
    M0 = G.nodes[t0]['M']

    return v0, delta0, M 

In [9]:
# TODO: perhaps the above should be split into a family of functions. Maybe a market object G 
# can have a callable ie. G.populate(payoff) that executes population of all node attributes but
# does not actually print the prices. Price printing can be done after population via
# separate functions. Perhaps when we do this we may want to make sure each possible payoff has a
# different name in case we want to have a portfolio that includes a variety of derivatives? 

In [10]:
K = 5. 

# European call with strike price K and underlying price s
def euro_call_payoff(s, K = K): 
    return max(0,s-K)

# European put with strike price K and underlying price s
def euro_put_payoff(s, K = K): 
    return max(0,K-s)

# payoff from being long in a futures contract
def futures_long_payoff(s, K = K):
    return s-K

# payoff from being short in a futures contract
def futures_short_payoff(s, K = K):
    return K-s

In [11]:
start=time.time()

euro_price, euro_delta, M = price_derivative(0, euro_call_payoff)

end=time.time()
runtime=end-start

print('Runtime = ', runtime, 's')

print('Euro put price at time 0 =  % .2f '% euro_price)
print('Number of shares in replicating portfolio at time 0 = % .2f ' % euro_delta)
print('Total bond investment in replicating portfolio at time 0 = % .2f' % M)

Runtime =  0.00012087821960449219 s
Euro put price at time 0 =   1.20 
Number of shares in replicating portfolio at time 0 =  0.50 
Total bond investment in replicating portfolio at time 0 = -0.80


In [12]:
# TODO: 
# - inspection of the implementation's performance for large number of periods? Numerical
# stability? 
# - effective visualization of results? 
# - Writing a baby package (PyCharm project?) that gives a user-friendly way to define markets
# and get quick answers for derivative pricing, hedging, etc without having to do anything yucky
# ie. without knowing numpy or networkx